In [71]:
# Schoof's algorithm v.1

# TO-DO:
#  - replace (x^3 + a*x + b) with f everywhere


# global variables:
q, a, b = 11, 2, 3
F_q = GF(q)
R.<x> = PolynomialRing(F_q)
S.<x1,y1> = GF(q)[]


def get_s_bar(l):  # pg.47
    q_l = q % l
    if q_l == 1:
        return x^(q^2) - x
    
    f_ql = get_f(q_l)
    f_ql_minus1 = get_f(q_l - 1)
    f_ql_plus1 = get_f(q_l + 1)
    
    if q_l % 2 == 0:
        res = 4*(x^(q^2)-x)*(x^3 + a*x + b)*f_ql^2 + f_ql_minus1*f_ql_plus1
    else:
        res = (x^(q^2)-x)*f_ql^2 + 4*(x^3 + a*x + b)*f_ql_minus1*f_ql_plus1
    return res

def get_f_bar(m):  # pg.38
    #m = ZZ(m)
    if m == 0:
        return 0
    elif m == 1:
        return 1
    elif m == 2:
        return 1
    elif m == 3:
        return 3*x^4 + 6*a*x^2 + 12*b*x - a^2
    elif m == 4:
        return 2*(x^6 + 5*a*x^4 + 20*b*x^3 - 5*a^2*x^2 - 4*a*b*x - 8*b^2 - a^3)
    else:
        m_bar = m // 2
        f_plus0 = get_f_bar(m_bar)
        f_plus1 = get_f_bar(m_bar + 1)
        f_plus2 = get_f_bar(m_bar + 2)
        f_minus1 = get_f_bar(m_bar - 1)
        
        if m % 2 == 1:
            if m_bar % 2 == 1:
                res = f_plus2*f_plus2^3 - 16*(x^3 + a*x + b)^2*f_minus1*f_plus1^3
            else:
                res = 16*(x^3 + a*x + b)^2*f_plus2*f_plus0^3 - f_minus1*f_plus1^3
        else:
            f_minus2 = get_f_bar(m_bar - 2)
            res = f_plus0*(f_plus2*f_minus1^2 - f_minus2*f_plus1^2)
        return res
                

def get_s(m):
    if m == 1:
        return 1
    
    f_plus0 = get_f_bar(m)
    #print(type(m))
    #print(type(ZZ(2)))
    m = ZZ(m)
    if m % 2 == 0:
        res = 16*(x^3 + a*x + b)^2*f_plus0^3
    else:
        res = f_plus0^3
        
    return res

def get_r(m):
    if m == 1:
        return 1
    
    f_plus1 = get_f_bar(m + 1)
    f_plus2 = get_f_bar(m + 2)
    f_minus1 = get_f_bar(m - 1)
    f_minus2 = get_f_bar(m - 2)
    
    res = f_plus2*f_minus1^2 - f_minus2*f_plus1^2  # incorrect during lecture?
    
    return res
        
def tyzero(l, q_l):
    s_ql = get_s(q_l)
    r_ql = get_r(q_l)
    mixed = s_ql*(x^3 + a*x + b)^((q^2 - 1)/2) + r_ql  # via lecture 14 - observation
    f_bar = get_f_bar(l)
    
    if gcd(f_bar, mixed) == 1:
        return False
    else:
        return True

def eigen(l, gamma, g_l):  # via lecture 14 - fact
    # 1st coordinates equal (we know from the "if" branch of schoof_core) ?
    
    s_gamma = get_s(gamma)
    r_gamma = get_r(gamma)
    mixed = (x^3 + a*x + b)^((q - 1)/2)*s_gamma - r_gamma
    if gcd(g_l, mixed) == 1:
        return False
    else:
        return True

def equalx(l, g_l):
    #q = params["q"]
    #F_q = params["F_q"]
    F_l = GF(l)
    
    #q_l = q % l
    q_l = F_l(q)
    if tyzero(l, q_l):
        return 0
    #tau = sqrt(F_l(4 * q_l))
    tau = sqrt(4 * q_l)  # over F_l
    gamma = 2 * q_l / tau  # over F_l
    if eigen(l, gamma, g_l):
        return tau
    return - tau

def get_c(m):
    if m == 1:
        return 0
    
    f_plus1 = get_f_bar(m + 1)
    f_minus1 = get_f_bar(m - 1)
    
    if m % 2 == 1:
        res = 4*(x^3 + a*x + b)*f_minus1*f_plus1
    else:
        res = f_minus1*f_plus1
    return res

def get_d(m):
    if m == 1:
        return 1
    
    f_plus0 = get_f_bar(m)
    
    if m % 2 == 1:
        res = f_plus0^2
    else:
        res = 4*(x^3 + a*x + b)*f_plus0^2

def get_h_X(l, tau, f_l_bar):  # fujhnus
    q_l = q % l
    
    c_ql = get_c(q_l)
    d_ql = get_d(q_l)
    r_ql = get_r(q_l)
    s_ql = get_s(q_l)
    
    c_tau = get_c(tau)
    d_tau = get_d(tau)
    
    den0 = s_ql^2*(d_ql*(x^(q^2) - x) + c_ql)^2 % f_l_bar
    den1 = d_ql
    den2 = d_tau(x^q) % f_l_bar
    
    den01 = den0 * den1
    den02 = den0 * den2
    den12 = den1 * den2
    
    den012 = den01 * den2
    
    num0 = (x^3 + a*x + b)*d_ql^2*((x^3 + a*x + b)^((q^2 - 1)/2) *s_ql - r_ql)^2 * den12 % f_l_bar
    num1 = c_ql * den02 % f_l_bar
    num2 = c_tau(x^q) * den01 % f_l_bar
    rest = (-x^(q^2)-x-x^q)*den012 % f_l_bar
    
    final = num0 + rest + num1 - num2
    
    return final

def get_h_Y(l, tau, f_l_bar):
    q_l = q % l
    
    c_ql = get_c(q_l)
    d_ql = get_d(q_l)
    r_ql = get_r(q_l)
    s_ql = get_s(q_l)
    
    s_tau = get_s(tau)
    r_tau = get_r(tau)
    
    den0 = s_ql*(d_ql*(x^(q^2) - x) + c_ql) % f_l_bar
    den1a = den0^2 % f_l_bar
    den1b = d_ql
    den1 = den1a * den1b % f_l_bar
    den2 = s_tau(x^q)
    
    den01 = den0 * den1
    #den02 = den0 * den2
    #den12 = den1 * den2
    
    den012 = den01 * den2
    
    smth0 = d_ql*((x^3 + a*x + b)^((q^2 - 1)/2) *s_ql - r_ql) % f_l_bar
    #smth0 = blah * den12 % f_l_bar
    smth1 = (2*x^(q^2) + x) * den1 - ((x^3 + a*x + b)*smth0^2)*den1b - c_ql*den1a
    smth1 = smth1 * den2 % f_l_bar
    smth2 = (x^3 + a*x + b)^((q - 1)/2) * r_tau(x^q) * den01 % f_l_bar
    rest = (x^3 + a*x + b)^((q^2 - 1)/2) * den012 % f_l_bar
    
    final = smth0 * smth1 - rest + smth2 % f_l_bar
    
    return final

def nonequalx(l, tau):
    f_l_bar = get_f_bar(l)
    
    h_X = get_h_X(l, tau, f_l_bar)
    h_Y = get_h_Y(l, tau, f_l_bar)
    
    
    if gcd(h_X, f_l_bar) == 1:
        return 0
    if gcd(h_Y, f_l_bar) == 1:
        return -1
    return 1

def recover_t(M):
    rems = []
    mods = []
    for pair in M:
        rems.append(pair[1])
        mods.append(pair[0])
    res = CRT_list(rems, mods)
    return res

def schoof_core():
    
    P = Primes()
    
    B = 2
    l = 2
    w_pol_x = x^3 + a*x + b
    #gcd_1 = gcd(t^q-t, w_pol_x)
    #print(gcd_1)
    if gcd(x^q-x, w_pol_x) == 1:
        tau = 1
    else:
        tau = 0
    M = [(2, tau)]
    while B < 4 * sqrt(RR(q)):  # is the sqrt over reals?
        l = P.next(l)
        print("Processing prime:", l)
        B *= l
        s_l_bar = get_s_bar(l)
        f_l_bar = get_f_bar(l)
        
        g_l = gcd(s_l_bar, f_l_bar)
        if g_l != 1:
            tau = equalx(l, g_l)
        else:
            tau = 0
            r = 0
            while r == 0:
                tau += 1
                r = nonequalx(l, tau)
            if r == -1:
                tau = - tau
        M.append((l, tau))
    t = recover_t(M)
    size = q + 1 - t
    return size

#my_q, my_a, my_b = 5, 2, 2
#schoof_core(my_q, my_a, my_b)

print(schoof_core())




# playground:
F_q = GF(7)
R.<t> = PolynomialRing(F_q)
#S.<x,y,z> = GF(5)[]
S = PolynomialRing(GF(7), 2, 'z', order='lex')  # invlex, neglex, lex
print(S)
#S.order = 'lex'
ff = z1 + z0^2
print(ff)
#print(ff(z0, z1, z2))
print(ff.reduce([z0^2-(z1^3)]))
#print(ff.degree(z))

f = t^2+2*t
f_2 = t^2+t
print(f_2.degree())

#ff = sqrt(F_q(2))
#print(ff)

Processing prime: 3


TypeError: not in prime subfield